Segmenting and Clustering Neighbourhoods in Toronto
============================

Prerequisites
------
Ensure that you have installed `BeautifulSoup` package from pypi using the below command 
`pip install BeautifulSoup`

Once BeautifulSoup is installed import all necessary libraries for the analyisis

In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

Get the html content from the wikipedia url given in the Assignment

In [2]:
html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup=BeautifulSoup(html.content)
table=soup.find('table')
rows=table.findAll('tr')
table_data={}
ix=0
for r in rows:
    row_data=r.findAll('td')
    if row_data:
        #assume that there are only 3 columns
        table_data.update({
            ix:[
                row_data[0].text, 
                row_data[1].text, 
                row_data[2].text.replace("\n",""),
            ]
        })
        ix+=1

In [3]:
columns=['Postal Code', 'Borough', 'Neighbourhood']

Create the dataframe from the initial data

In [4]:
df = pd.DataFrame.from_dict(data=table_data, orient='index', columns=columns)

In [5]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Clean up the dataframe and create a new data frame as per the given conditions in point No3 of the assignment

In [6]:
new_data={}
for index, row in df.iterrows():
    if row['Borough']=='Not assigned':
        continue
    
    if row['Neighbourhood']=='Not assigned':
        row['Neighbourhood']=row['Borough']
        
    if row['Postal Code'] in new_data.keys():
        if row['Neighbourhood'] not in new_data[row['Postal Code']][2]:
            row['Neighbourhood'] = new_data[row['Postal Code']][2] + " , " + row['Neighbourhood']
        
    new_data.update({
        row['Postal Code']: [row[0],row[1],row[2]]
    })
    
# steps to create dictionary for the DataFrame
new_data_new={}
xcount=0
for i in new_data.keys():
    new_data_new.update({
        xcount: new_data[i]
    })
    xcount+=1
new_df=pd.DataFrame.from_dict(data=new_data_new, orient='index', columns=columns)

In [7]:
new_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront , Regent Park"
3,M6A,North York,"Lawrence Heights , Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [8]:
new_df.shape

(103, 3)

In [9]:
import geocoder

In [10]:
def get_lat_long(postal_code, neighbourhood):
    latlng_coords=None
    while (latlng_coords is None):
        g=geocoder.google('{}, {}'.format(postal_code, neighbourhood))
        latlng_coords= g.latlng
        
    return latlng_coords

In [11]:
# Not reliable get_lat_long('M5G','Harbourfront , Regent Park')

As the Method written above is not reliable to fe

In [13]:
latlng_coords=pd.read_csv('Geospatial_Coordinates.csv')

In [14]:
latlng_coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
latlng_coords.shape

(103, 3)

In [16]:
combined_df=pd.merge(new_df,latlng_coords,on='Postal Code')

In [17]:
combined_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront , Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights , Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
